In [34]:
# Required packages.
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import np_utils
from keras import optimizers

In [35]:
# fix random seed for reproducibility.
seed = 7
np.random.seed(seed)

In [36]:
# Load data.
dataset = np.loadtxt("data_tp1", delimiter=",")
X_train = dataset[:, 1:].reshape(dataset.shape[0],1,28, 28).astype('float32')
y_train = dataset[:,0]
X_test = dataset[:,1:].reshape(dataset.shape[0],1, 28, 28).astype('float32')
y_test = dataset[:,0]

In [37]:
# An image is represented by 28*28 then it is a 784 in memory.
num_pixels = 784
X_train = X_train.reshape(X_train.shape[0], num_pixels).astype('float32')
X_test = X_test.reshape(X_test.shape[0], num_pixels).astype('float32')

In [38]:
# Normalize inputs from 0-255 to 0-1.
X_train = X_train / 255
X_test = X_test / 255

In [39]:
# One hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [40]:
# Define model
def baseline_model(unity, learning_rate):
    model = Sequential()
    model.add(Dense(num_pixels, input_dim=num_pixels, kernel_initializer='normal', activation='relu'))
    model.add(Dense(25, input_dim=unity, kernel_initializer='normal', activation='relu'))
    model.add(Dense(num_classes, kernel_initializer='normal', activation='softmax'))
    # Set the gradient
    sgd = optimizers.SGD(lr=learning_rate)
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    return model

In [45]:
# Set the variables.
unity = 100
learning_rate = 0.5

In [46]:
# Build the model.
model = baseline_model(unity, learning_rate)
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200, verbose=2)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Training Error: %.2f%%" % (100-scores[1]*100))

Train on 5000 samples, validate on 5000 samples
Epoch 1/10
 - 1s - loss: 2.2963 - acc: 0.1094 - val_loss: 2.2690 - val_acc: 0.2120
Epoch 2/10
 - 0s - loss: 2.2690 - acc: 0.2120 - val_loss: 2.2365 - val_acc: 0.2966
Epoch 3/10
 - 0s - loss: 2.2365 - acc: 0.2966 - val_loss: 2.1921 - val_acc: 0.3584
Epoch 4/10
 - 0s - loss: 2.1921 - acc: 0.3584 - val_loss: 2.1289 - val_acc: 0.4018
Epoch 5/10
 - 0s - loss: 2.1289 - acc: 0.4018 - val_loss: 2.0396 - val_acc: 0.4292
Epoch 6/10
 - 0s - loss: 2.0396 - acc: 0.4292 - val_loss: 1.9197 - val_acc: 0.4632
Epoch 7/10
 - 0s - loss: 1.9197 - acc: 0.4632 - val_loss: 1.7716 - val_acc: 0.5184
Epoch 8/10
 - 0s - loss: 1.7716 - acc: 0.5184 - val_loss: 1.6081 - val_acc: 0.5890
Epoch 9/10
 - 0s - loss: 1.6081 - acc: 0.5890 - val_loss: 1.4442 - val_acc: 0.6598
Epoch 10/10
 - 1s - loss: 1.4442 - acc: 0.6598 - val_loss: 1.2902 - val_acc: 0.7120
Training Error: 28.80%
